In [2]:
# Housekeeping
import requests
import datetime
import pandas as pd
import io
import os
import trueskill

import oeutils

pd.options.display.max_rows = 100
pd.options.display.max_columns = 500

workingdir = '../data/'
years = ['2018', '2019', '2020', '2021']

#oeutils.download(workingdir, years)
df = oeutils.read('../data/')
df = oeutils.clean(df, split_on='team', keep_identities=False, keep_leagues=False, keep_columns=False)

/home/paulperrone/source/lol-analysis/notebooks/oeutils.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oe_data['gameid'] = oe_data['gameid'].str.strip()


In [3]:
df.shape

(37246, 117)

In [4]:
df['league'].unique()

array(['KeSPA', 'LPL', 'LFL', 'LEC', 'LCS.A', 'CBLOL', 'LCS', 'UL', 'OPL',
       'VCS', 'TCL', 'TRA', 'BRCC', 'UKLC', 'UPL', 'LCK', 'CK', 'LJL',
       'LLA', 'LCL', 'PCS', 'EM', 'LDL', 'BIG', 'OCS', 'Riot', 'MSC',
       'RCL', 'CU', 'NLC', 'WCS', 'NEST', 'NASG', 'DC', 'EU LCS', 'LLN',
       'LMS', 'CLS', 'NA LCS', 'TCS', 'CIS CL', 'GPL', 'MSI'],
      dtype=object)

In [5]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
10,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,100,Blue,team,NaN,SANDBOX Gaming,NaN,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,2220,1,17,11,33,17,11,3.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.4595,0.7568,0.0,2.0,3.0,2.0,3.0,0.0,2.0,0.0,0.0,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,8.0,4.0,0.0,0.0,1.0,0.0,77679,2099.4324,NaN,1975.3243,2302.4595,191,5.1622,97,2.6216,67,380.0,10.2703,68927,44905,1213.6486,NaN,62275.0,0.004023,NaN,1049.0,216,166,0,34.1892,14615.0,17845.0,330.0,14498.0,17982.0,322.0,117.0,-137.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,21953.0,27219.0,507.0,22778.0,28884.0,519.0,-825.0,-1665.0,-12.0,0.0,0.0,1.0,1.0,2.0,0.0
11,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,200,Red,team,NaN,T1,NaN,Lucian,Akali,Lee Sin,Olaf,Jarvan IV,2220,0,11,17,25,11,17,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.2973,0.7568,1.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,8.0,1.0,1.0,0.0,1.0,48685,1315.8108,NaN,2934.9459,3135.6486,172,4.6486,91,2.4595,69,318.0,8.5946,62709,38687,1045.5946,NaN,62025.0,-0.004023,NaN,1021.0,271,193,5,34.9189,14498.0,17982.0,322.0,14615.0,17845.0,330.0,-117.0,137.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,22778.0,28884.0,519.0,21953.0,27219.0,507.0,825.0,1665.0,12.0,1.0,2.0,0.0,0.0,0.0,1.0
22,ESPORTSTMNT03/1241322,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 09:00:58,2,9.24,100,Blue,team,NaN,T1,NaN,Syndra,LeBlanc,Rumble,Braum,Leona,2227,0,5,21,16,5,21,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.1347,0.7005,0.0,1.0,4.0,1.0,4.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,9.0,1.0,0.0,0.0,1.0,72963,1965.7746,NaN,2324.1042,2729.5016,144,3.8797,95,2.5595,58,254.0,6.8433,57283,33190,894.2075,NaN,56575.0,-0.122771,NaN,1011.0,183,151,1,32.1688,14180.0,16947.0,300.0,16194.0,19096.0,334.0,-2014.0,-2149.0,-34.0,0.0,0.0,3.0,3.0,3.0,0.0,21714.0,26869.0,501.0,27198.0,31702.0,557.0,-5484.0,-4833.0,-56.0,0.0,0.0,6.0,6.0,7.0,0.0
23,ESPORTSTMNT03/1241322,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 09:00:58,2,9.24,200,Red,team,NaN,SANDBOX Gaming,NaN,Lucian,Akali,Rek'Sai,Irelia,Camille,2227,1,21,5,41,21,5,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.5658,0.7005,1.0,4.0,1.0,4.0,1.0,0.0,1.0,1.0,2.0,NaN,0.0,0.0,1.0,2.0,0.0,1.0,2.0,0.0,1.0,9.0,2.0,0.0,1.0,1.0,0.0,65542,1765.8374,NaN,2861.2214,3192.7167,195,5.2537,90,2.4248,53,397.0,10.6960,72004,47911,1290.8217,NaN,63975.0,0.122771,NaN,976.0,270,126,48,33.5698,16194.0,19096.0,334.0,14180.0,16947.0,300.0,2014.0,2149.0,34.0,3.0,3.0,0.0,0.0,0.0,3.0,27198.0,31702.0,557.0,21714.0,26869.0,501.0,5484.0,4833.0,56.0,6.0,7.0,0.0,0.0,0.0,6.0
34

In [6]:
lcs = df[df['league'].isin(['LCS', 'NA LCS'])]
lcs.shape

(1480, 117)

In [7]:
lcs['team'].unique()

array(['Cloud9', 'Team Liquid', 'Counter Logic Gaming', 'Dignitas',
       '100 Thieves', 'Golden Guardians', 'Immortals', 'FlyQuest',
       'Evil Geniuses', 'TSM', 'OpTic Gaming', 'Clutch Gaming',
       'Echo Fox'], dtype=object)

In [8]:
lcs = lcs.sort_values(by=['date', 'team'])

In [10]:
team_map = {
    'OpTic Gaming': 'Immortals',
    'Echo Fox': 'Evil Geniuses',
    'Clutch Gaming':'Dignitas'
}

def replace_team_name(team_to_replace):
    if team_to_replace in team_map.keys():
        return team_map[team_to_replace]
    else:
        return team_to_replace

lcs['team'] = lcs.apply(lambda row: replace_team_name(row['team']), axis=1)
lcs['team'].unique()

array(['TSM', 'Team Liquid', '100 Thieves', 'Immortals', 'Dignitas',
       'Golden Guardians', 'Evil Geniuses', 'FlyQuest', 'Cloud9',
       'Counter Logic Gaming'], dtype=object)

In [11]:
lcs.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
1703,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,200,Red,team,NaN,TSM,NaN,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,1,11,2,1,11,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0356,0.4268,1.0,2.0,2.0,2.0,2.0,0.0,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,10.0,0.0,0.0,0.0,1.0,27569,980.5216,NaN,2336.5857,2197.7712,95,3.3788,38,1.3515,24,221.0,7.8601,47020,28435,1011.3219,NaN,44225.0,-0.160319,NaN,951.0,149,100,4,39.1227,15755.0,18703.0,374.0,15970.0,18541.0,341.0,-215.0,162.0,33.0,0.0,0.0,1.0,1.0,1.0,0.0,24118.0,29903.0,599.0,26505.0,31407.0,551.0,-2387.0,-1504.0,48.0,0.0,0.0,4.0,4.0,10.0,0.0
1702,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,100,Blue,team,NaN,Team Liquid,NaN,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,11,1,36,11,1,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.3912,0.4268,0.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,10.0,2.0,1.0,1.0,1.0,0.0,45826,1629.8518,NaN,1814.7955,2226.2952,114,4.0545,32,1.1381,36,224.0,7.9668,59283,40698,1447.4689,NaN,51933.0,0.160319,NaN,869.0,196,124,32,37.8779,15970.0,18541.0,341.0,15755.0,18703.0,374.0,215.0,-162.0,-33.0,1.0,1.0,0.0,0.0,0.0,1.0,26505.0,31407.0,551.0,24118.0,29903.0,599.0,2387.0,1504.0,-48.0,4.0,10.0,0.0,0.0,0.0,4.0
1714,TRLH1/1002440076,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 22:38:20,1,8.01,100,Blue,team,NaN,100 Thieves,NaN,Tahm Kench,Orianna,Kalista,Taric,Ornn,3873,1,9,7,27,9,7,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.1394,0.2479,1.0,5.0,1.0,3.0,0.0,0.0,1.0,0.0,2.0,NaN,2.0,1.0,0.0,0.0,1.0,1.0,3.0,2.0,0.0,11.0,7.0,0.0,0.0,3.0,1.0,112531,1743.3153,NaN,3514.5778,6596.7622,245,3.7955,144,2.2308,37,609.0,9.4345,120533,79650,1233.9272,NaN,112358.0,0.098992,NaN,1938.0,320,199,38,34.9806,15137.0,19223.0,344.0,16248.0,19671.0,351.0,-1111.0,-448.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,23061.0,30627.0,560.0,24670.0,31467.0,564.0,-1609.0,-840.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0
1715,TRLH1/1002440076,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 22:38:20,1,8.01,200,Red,team,NaN,Immortals,NaN,Zoe,Evelynn,Azir,Gangplank,Gnar,3873,0,7,9,17,7,9,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.1084,0.2479,0.0,1.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,1.0,2.0,1.0,1.0,0.0,0.0,2.0,3.0,1.0,7.0,11.0,1.0,1.0,1.0,3.0,187337,2902.1998,NaN,2460.2634,3162.9589,300,4.6476,101,1.5647,45,570.0,8.8304,114382,73499,1138.6367,NaN,101760.0,-0.098992,NaN,1718.0,362,238,29,32.2231,16248.0,19671.0,351.0,15137.0,19223.0,344.0,1111.0,448.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,24670.0,31467.0,564.0,23061.0,30627.0,560.0,1609.0,840.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1726,

In [12]:
input_data = lcs[['gameid', 'date', 'playerid', 'side', 'team', 'result']]
input_data.head()

,gameid,date,playerid,side,team,result
1703,TRLH1/1002440062,2018-01-20 20:50:09,200,Red,TSM,0
1702,TRLH1/1002440062,2018-01-20 20:50:09,100,Blue,Team Liquid,1
1714,TRLH1/1002440076,2018-01-20 22:38:20,100,Blue,100 Thieves,1
1715,TRLH1/1002440076,2018-01-20 22:38:20,200,Red,Immortals,0
1726,TRLH1/1002440084,2018-01-21 00:14:20,100,Blue,Dignitas,1


In [21]:
starting_elo = 1000
columns = ['gameid', 'date', \
           'blue_team', 'red_team', \
           ''
           'blue_team_win_prediction', 'blue_team_result', \
           'Cloud9_start_rating', 'Team_Liquid_start_rating', 'Counter_Logic_Gaming_start_rating', 'Dignitas_start_rating', \
           '100_Thieves_start_rating', 'Golden_Guardians_start_rating', 'Immortals_start_rating', 'FlyQuest_start_rating', \
           'Evil_Geniuses_start_rating', 'TSM_start_rating', \
           'Cloud9_exit_rating', 'Team_Liquid_exit_rating', 'Counter_Logic_Gaming_exit_rating', 'Dignitas_exit_rating', \
           '100_Thieves_exit_rating', 'Golden_Guardians_exit_rating', 'Immortals_exit_rating', 'FlyQuest_exit_rating', \
           'Evil_Geniuses_exit_rating', 'TSM_exit_rating']

lcs_rating = pd.DataFrame(columns=columns)

In [22]:
starter_row = pd.Series([0, datetime.datetime(2018, 1, 1), \
               '0','0', \
               0,0, \
               1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, \
               1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000], index=lcs_rating.columns)

lcs_rating = lcs_rating.append(starter_row, ignore_index=True)
lcs_rating.head()

,gameid,date,blue_team,red_team,blue_team_win_prediction,blue_team_result,Cloud9_start_rating,Team_Liquid_start_rating,Counter_Logic_Gaming_start_rating,Dignitas_start_rating,100_Thieves_start_rating,Golden_Guardians_start_rating,Immortals_start_rating,FlyQuest_start_rating,Evil_Geniuses_start_rating,TSM_start_rating,Cloud9_exit_rating,Team_Liquid_exit_rating,Counter_Logic_Gaming_exit_rating,Dignitas_exit_rating,100_Thieves_exit_rating,Golden_Guardians_exit_rating,Immortals_exit_rating,FlyQuest_exit_rating,Evil_Geniuses_exit_rating,TSM_exit_rating
0,0,2018-01-01,0,0,0,0,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000


## Trueskill

In [28]:
ts = trueskill.TrueSkill(draw_probability=0.0)

In [29]:
rating_dict = dict()
for i in input_data['team'].unique():
    rating_dict[i] = ts.create_rating()
    
print(rating_dict)

{'TSM': trueskill.Rating(mu=25.000, sigma=8.333), 'Team Liquid': trueskill.Rating(mu=25.000, sigma=8.333), '100 Thieves': trueskill.Rating(mu=25.000, sigma=8.333), 'Immortals': trueskill.Rating(mu=25.000, sigma=8.333), 'Dignitas': trueskill.Rating(mu=25.000, sigma=8.333), 'Golden Guardians': trueskill.Rating(mu=25.000, sigma=8.333), 'Evil Geniuses': trueskill.Rating(mu=25.000, sigma=8.333), 'FlyQuest': trueskill.Rating(mu=25.000, sigma=8.333), 'Cloud9': trueskill.Rating(mu=25.000, sigma=8.333), 'Counter Logic Gaming': trueskill.Rating(mu=25.000, sigma=8.333)}
